In [ ]:
from datetime import datetime
import getpass
import requests
import time

from astropy import units as u
import numpy as np

import lsst.verify

In [ ]:
METRIC_PACKAGE = "sysver_metrics"
METRIC_NAME = "camera.CameraBodySurfaceTemp"
SQUASH_API_URL = "https://squash-restful-api-sandbox.lsst.codes"

In [ ]:
username = getpass.getuser()
password = getpass.getpass(prompt='Password for user `{}`: '.format(username))

In [ ]:
credentials = {'username': username, 'password': password}
r = requests.post('{}/auth'.format(SQUASH_API_URL), json=credentials)
#r.json()

In [ ]:
headers = {'Authorization': 'JWT {}'.format(r.json()['access_token'])}

In [ ]:
# Load the entire package
sysver_metrics = lsst.verify.MetricSet.load_metrics_package(METRIC_PACKAGE)
#sysver_metrics
r = requests.post('{}/metrics'.format(SQUASH_API_URL), json={'metrics': sysver_metrics.json}, headers=headers)
print(r.json())

In [ ]:
# This is how specifications for metrics are read
sysver_specs = lsst.verify.SpecificationSet.load_metrics_package(METRIC_PACKAGE)
#sysver_specs
r = requests.post('{}/specs'.format(SQUASH_API_URL), json={'specs': sysver_specs.json}, headers=headers)
print(r.json())

In [ ]:
def measure_temperature(temperature_offset=None, percentage=0.1):
    # Parameters for a Gaussian temparature distribution
    mu = 10 # Celsius
    sigma = 0.1 # Celsius
    num_measurements = 1000

    temp = np.random.normal(mu, sigma, num_measurements) * u.deg_C
    
    if temperature_offset is not None:
        frac = int(percentage * num_measurements)
        value = np.random.randint(0, num_measurements, frac)
        indexes = np.unique(value)
        temp[indexes] += temperature_offset * u.deg_C
    
    avg_temp = np.mean(temp)
    return avg_temp, temp

In [ ]:
do_ramp = False
iteration = 1
temperature_ramp = 0.75
fraction_ramp = 0.4
temperature_offset = temperature_ramp * iteration if do_ramp else None
fraction_change = fraction_ramp * iteration if do_ramp else fraction_ramp

In [ ]:
average_temperature, temperature_data = measure_temperature(temperature_offset, fraction_change)
positions = np.arange(0, 1000, 20) * u.millimeter
angles = np.arange(0, 360.0, 18.0) * u.deg
body_position, body_angle = np.meshgrid(positions, angles)
avg_temp_meas = lsst.verify.Measurement(METRIC_NAME, average_temperature)
avg_temp_meas.notes['estimator'] = 'numpy.mean'
avg_temp_meas.extras['cameraBody_temperature_grid'] = lsst.verify.Datum(temperature_data,
                                                                        label='Surface Temperature',
                                                                        description='Grid of Camera body surface temperatures')
avg_temp_meas.extras['cameraBody_positions'] = lsst.verify.Datum(body_position.flatten(),
                                                                 label='Body Position',
                                                                 description='Location of sensor positions along Camera body')
avg_temp_meas.extras['cameraBody_angles'] = lsst.verify.Datum(body_angle.flatten(),
                                                              label='Body Angle',
                                                              description='Location of sensor positions along Camera circumference')

In [ ]:
job = lsst.verify.Job()
job.metrics.update(sysver_metrics)
job.specs.update(sysver_specs)
job.measurements.insert(avg_temp_meas)
outfile = "sysver_{}.json".format(datetime.now().strftime("%Y%m%d_%H%M%S"))
job.write(outfile)

In [ ]:
#job.dispatch(api_user=username, api_password=password, api_url=SQUASH_API_URL)

In [ ]:
%%bash -s "$SQUASH_API_URL" "$username" "$password" "$outfile"
export DYLD_LIBRARY_PATH=$LSST_LIBRARY_PATH
dispatch_verify.py --ignore-lsstsw --url $1 --user $2 --password $3 $4